# Concurso Público para a BB Tecnologia e Serviços - Classificação Preliminar (Analista - Perfil Tecnológico)

Até a data de hoje, 02 de Novembro de 2023, a banca examinadora FGV, encarregada de organizar o referido concurso público, já deu uma classificação preliminar de seus candidatos aprovados com base nas notas das provas objetivas, e também corrigiu as provas discursivas. Entretando, não soltaram na página de acompanhamento (https://conhecimento.fgv.br/concursos/bbts23) nenhum arquivo com a classificação preliminar dos candidatos levando em conta a pontuação das provas discursivas. 

O objetivo deste notebook é criar um arquivo PDF com a classificação preliminar dos candidatos.  

Antes de prosseguirmos com este trabalho, é importante salientar os seguintes pontos: 
 - O resultado deste trabalho, o arquivo PDF, não é um documento oficial, mas apenas um estudo, e não deve ser utilizado, em hipótese alguma, para qualquer fim que não sanar alguma curiosidade;
 - Todos os dados utilizados e divulgados com este trabalho são públicos e podem facilmente ser encontrados no link referido acima;
 - Não foram aplicados critérios de desempate.

# Bibliotecas utilizadas:

- Pandas: Para manipulação de dados.
- Tabula: Para extração de dados de tabelas em arquivos PDF.
- ReportLab: Para criação do arquivo PDF com a classificação preliminar.  

In [1]:
import pandas as pd 
import tabula as tb # para extração de dados de tabelas em arquivos PDF


#-----------#   para montar o arquivo pdf ao final
#--------------------------------------------------------#
from reportlab.lib import colors
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, PageBreak, Paragraph
from reportlab.lib.styles import getSampleStyleSheet
#--------------------------------------------------------#

# Dados:

## Notas das provas objetivas: 
Os dados das provas objetivas foram extraídos do arquivo em PDF encontrado no seguinte link: 

https://conhecimento.fgv.br/sites/default/files/concursos/423_bb_tec.-e-serv-resultado-definitivo-prova-objetiva-2023-09-29.pdf

Este arquivo foi baixado e editado previamente usando a ferramenta de linha de comando `pdftk`, para que fossem removidas as páginas que não contivessem informações sobre os candidatos ao cargo de Analista - Perfil Tecnológico, objeto deste trabalho. O resultado da edição foi salvo no arquivo `aprovados.pdf`.

### Compilação do DataFrame:

In [2]:
objetivas = tb.read_pdf('aprovados.pdf', pages='all')

for df in objetivas:
    df.drop(df.columns[2], axis=1, inplace=True)
    df.columns = df.loc[0]
    df.drop(0, axis=0, inplace=True)
    
objetivas = pd.concat([i for i in objetivas], ignore_index=True)
objetivas.columns = ['id', 'nome', 'port', 'raciocinio', 'mat', 'est', 'gerais', 'ingles','dsc','dev', 'bd', 'infra','sec','nota','status']

numeric_columns = ['port', 'raciocinio', 'mat', 'est', 'gerais', 'ingles','dsc','dev', 'bd', 'infra','sec','nota']
for col in numeric_columns:
    objetivas[col] = pd.to_numeric(objetivas[col])

objetivas['nota'] = objetivas['nota'].apply(lambda x: int(x))

objetivas

,id,nome,port,raciocinio,mat,est,gerais,ingles,dsc,dev,bd,infra,sec,nota,status
0,423012294,Vinicius De Santanna Salustiano,8,9,8,4,3,6,3,5,3,3,2,54,Aprovado
1,423073103,Guilherme Teles Da Mota,5,10,9,4,5,4,3,5,3,1,3,52,Aprovado Negro
2,423042504,Matheus Antunes Corinos Lima,6,8,10,4,5,5,2,3,2,3,3,51,Aprovado
3,423078536,Fabio Luiz Da Rocha Moraes,6,8,9,3,1,6,4,3,4,3,3,50,Aprovado
4,423075138,Rodrigo Maestrali Almeida,7,9,7,4,4,5,2,3,3,2,4,50,Aprovado
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,423006626,Renan Maia Behr Da Rocha,7,5,4,3,3,3,3,1,2,3,1,35,Aprovado
130,423004502,Elaine Silva Barbosa,7,3,6,4,2,3,2,2,2,2,2,35,Aprovado
131,423078859,Marcus Vinicius Celião Da Silva,5,8,4,3,3,4,1,0,3,2,2,35,Aprovado
132,423057281,Pablio Martins Dos Anjos Souza,7,7,5,2,2,3,2,0,3,2,2,35,Aprovado


## Notas das provas discursivas: 
Os dados das provas discursivas podem ser encontrados no seguinte link: 

https://conhecimento.fgv.br/sites/default/files/concursos/423_bb_tec.-e-serv-resultado-preliminar-prova-discursiva-2023-10-09_0.pdf

Não foi possível extrair estes dados do arquivo PDF através da biblioteca Tabula, de modo que foi necessário copiar as linhas da tabela pertinente, colar em um arquivo de texto, salvar em formato `.csv`, importar com a biblioteca Pandas e formatá-los como se vê a seguir.

In [3]:
discursivas  = pd.read_csv('redacao.csv', delimiter=';', names=['nome'])

### Funções utilitárias:

In [4]:
def format_status(row):
    if row['nome'][-8:] == 'Aprovado':
        row['status'] = 'AC'
        row['nome'] = row['nome'][:-8]
    elif row['nome'][-5:] == 'Negro':
        row['status'] = 'PPP'
        row['nome'] = row['nome'][:-14]
    elif row['nome'][-3:] == 'PcD':
        row['status'] = 'PcD'
        row['nome'] = row['nome'][:-12]
    return row

def format_nota(row):
    if row['nome'][-2] == ',':
        row['nota'] = float(row['nome'][-4:].replace(',','.'))
        row['nome'] = row['nome'][:-4]
    else:
        row['nota'] = float(row['nome'][-2:])
        row['nome'] = row['nome'][:-2]
    return row

### Compilação do DataFrame:

In [5]:
discursivas['id'] = discursivas['nome'].apply(lambda x: x[:9])
discursivas['nome'] = discursivas['nome'].apply(lambda x: x[9:])
discursivas = discursivas.apply(format_status, axis=1)
discursivas = discursivas.apply(format_nota, axis=1)

discursivas

,nome,id,status,nota
0,Aerton Pinheiro Marzagão Filho,423021967,AC,27.2
1,Alexandre Falchi Pusaudse,423031874,PcD,28.7
2,Alvaro Henrique Figueiredo Nunes,423046915,AC,28.8
3,Amauri Da Costa Júnior,423056912,AC,26.1
4,André Gomes Ferreira De Oliveira,423050986,PPP,27.4
...,...,...,...,...
78,Vinicius De Santanna Salustiano,423012294,AC,28.1
79,Vitor Faria Assi,423054327,AC,28.7
80,Vitor Vasconcelos De Oliveira,423034718,AC,28.2
81,Warley Moura Do Nascimento,423027963,AC,28.8


# Classificação dos Candidatos: 

Abaixo, os DataFrames `discursivas` e `objetivas` são usados para gerar o DataFrame `geral`, que contem os dados dos candidatos que tiveram redação corrigida, com pontuação das provas objetivas e discursivas somadas, e já ordenados de acordo com suas notas. 

In [6]:
discursivas = discursivas.set_index('id').sort_index()
objetivas = objetivas.set_index('id').sort_index()

geral = objetivas.merge(discursivas, left_index=True, right_index=True)
geral['total'] = geral['nota_x'] + geral['nota_y']
geral = geral.drop(columns=['nome_y', 'status_x'], axis=1)
geral.rename(columns={'nome_x': 'nome', 'nota_x':'objetiva', 'nota_y':'discursiva', 'status_y':'status'}, inplace=True)
geral.sort_values(by='total', ascending=False, inplace=True)

## Separação dos candidatos de acordo com sistema de concorrência:
É gerada a lista de DataFrames `resultado_preliminar`, com DataFrames contendo os candidatos já classificados em seus respectivos sistemas de concorrência: Ampla Concorrência, Pretos e Pardos, e Pessoas com Deficiência.

In [7]:
resultado_preliminar = []

for label in geral.status.unique():
    df = geral[geral['status'] == label].reset_index().drop('status', axis=1)
    df.drop(df.columns[2:13], axis=1, inplace=True)
    posicao = [x for x in range(1, len(df)+1)]
        
    for i in range(2, len(df)):
        if df.iloc[i-1][-1] == df.iloc[i][-1]:
            posicao[i] = posicao[i-1]
    
    df['Posição'] = posicao
    df.columns = ['Inscrição', 'Nome', 'Prova Objetiva', 'Prova Discursiva', 'Total', 'Posição']
    colunas = ['Posição', 'Inscrição', 'Nome', 'Prova Objetiva', 'Prova Discursiva', 'Total']
    df = df[colunas]
    resultado_preliminar.append(df)

# Arquivo PDF:
 Abaixo, é gerado o arquivo PDF com as tabelas para cada sistema de concorrência. 

In [8]:
titulos = ['Ampla Concorrência', 'Pretos e Pardos', 'Pessoas com Deficiência']
arquivo = 'bbts_classificacao_preliminar.pdf'
doc = SimpleDocTemplate(arquivo, pagesize=(letter[1], letter[0]))
tabelas = []

for df, titulo in zip(resultado_preliminar, titulos):
    data = [df.columns[:,].values.astype(str)] + df.values.tolist()
    tabela = Table(data)

    estilo = TableStyle([('BACKGROUND', (0, 0), (-1, 0), colors.grey),
                        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
                        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
                        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
                        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
                        ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
                        ('GRID', (0, 0), (-1, -1), 1, colors.black)])

    tabela.setStyle(estilo)

    estilo_titulo = getSampleStyleSheet()['Title']
    paragrafo_titulo = Paragraph(titulo, estilo_titulo)

    tabelas.append(paragrafo_titulo)
    tabelas.append(tabela)
    tabelas.append(PageBreak())

# Build the PDF document with the list of tables
doc.build(tabelas)